In [1]:
import yfinance as yf
from pandas import *
import numpy as np
import pandas_ta as ta
import seaborn as sns
import matplotlib.pyplot as pglt
from datetime import datetime
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import math 

In [2]:
# inda = yf.download("INDA", start="2010-01-01", end="2025-3-31")
# inda.columns = [col[0] if isinstance(col, tuple) else col for col in inda.columns]
# inda = read_csv(r"INDA_Historical_Data.csv")
# inda['Date'] = to_datetime(inda['Date'], format='%d-%m-%Y')
# inda = inda.rename(columns={'Price':'Close', 'Vol.':'Volume'})
# inda= inda.set_index('Date').sort_index()
# def convert_volume(val):
#     if isinstance(val, str):
#         val = val.strip()
#         if val.endswith('M'):
#             return float(val[:-1]) * 1_000_000
#         elif val.endswith('K'):
#             return float(val[:-1]) * 1_000
#     return float(val)

# inda['Volume'] = inda['Volume'].apply(convert_volume)
# inda = inda.drop('Change %',axis=1)
# inda.columns = inda.columns.str.strip().str.lstrip('#').str.strip().str.lower()
# inda.ta.strategy("All",append=True)

In [3]:
df= read_csv(r"E:\Portfolio\Portfolio\INDA-CRISP\INDA_TI_ALL.csv")
df['Date']= to_datetime(df['Date'])
df.set_index('Date',inplace=True)
df

,close,open,high,low,volume,ABER_ZG_5_15,ABER_SG_5_15,ABER_XG_5_15,ABER_ATR_5_15,ACCBL_20,...,VIDYA_14,VTXP_14,VTXM_14,VWAP_D,VWMA_10,WCP,WILLR_14,WMA_10,ZL_EMA_10,ZS_30
Date,,,,,,,,,,,,,,,,,,,,,
2012-02-06,26.51,27.64,27.64,26.43,4050.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,26.860000,NaN,26.7725,NaN,NaN,NaN,NaN
2012-02-07,26.27,26.26,26.27,26.26,2300.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,26.266667,NaN,26.2675,NaN,NaN,NaN,NaN
2012-02-08,26.56,27.63,27.63,26.40,3650.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,26.863333,NaN,26.7875,NaN,NaN,NaN,NaN
2012-02-09,26.49,26.63,26.63,26.49,2550.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,26.536667,NaN,26.5250,NaN,NaN,NaN,NaN
2012-02-10,26.01,26.30,26.30,25.89,6200.0,26.518667,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,26.066667,NaN,26.0525,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-04-29,53.54,53.40,53.56,53.27,4080000.0,53.284000,54.083558,52.484442,0.799558,49.858730,...,51.525202,1.108629,0.644068,53.456667,52.499719,53.4775,-4.406130,53.140727,53.493361,1.588206
2025-04-30,53.57,53.46,53.78,53.28,7090000.0,53.334000,54.113587,52.554413,0.779587,49.963363,...,51.629043,1.163551,0.655140,53.543333,52.953722,53.5500,-5.172414,53.287636,53.476387,1.497545
2025-05-01,53.54,53.65,53.72,53.49,5260000.0,53.339333,54.082282,52.596385,0.742948,50.083793,...,51.768948,1.146341,0.603393,53.583333,53.182452,53.5725,-6.400000,53.384727,53.613407,1.366772


In [4]:
df = df.loc[:, df.isna().mean() <= 0.2]
first_valid_index = df.dropna().index[0]
df = df.loc[first_valid_index:]
df= df.truncate(before=datetime(2015,1,1))
nan_columns= df.columns[df.isna().any()]
df_nan_only= df[nan_columns]
df_nan_only.tail(10)

,volume,AD,OBV,OBV_min_2,OBV_max_2,CMF_20,DPO_20,EOM_14_100000000,ICS_26,MFI_14,PVOL,PVT,VWAP_D,VWMA_10
Date,,,,,,,,,,,,,,
2025-04-22,10240000.0,6.321368e+08,349488980.0,339248980.0,349488980.0,0.026331,NaN,1.367114,NaN,68.354310,547840000.0,-2.404893e+08,53.506667,51.356859
2025-04-23,7360000.0,6.266968e+08,342128980.0,342128980.0,349488980.0,-0.045529,NaN,1.091728,NaN,64.009535,391404800.0,-2.448915e+08,53.293333,51.643358
2025-04-24,4290000.0,6.305648e+08,346418980.0,342128980.0,346418980.0,0.017547,NaN,1.272104,NaN,69.961360,230544600.0,-2.403740e+08,53.556667,51.854973
2025-04-25,6680000.0,6.367310e+08,339738980.0,339738980.0,346418980.0,0.047062,NaN,1.753569,NaN,73.048311,353038000.0,-2.514369e+08,52.770000,52.081657
2025-04-28,7110000.0,6.344558e+08,346848980.0,339738980.0,346848980.0,0.064229,NaN,2.162750,NaN,82.722345,378891900.0,-2.455175e+08,53.343333,52.353656
2025-04-29,4080000.0,6.379730e+08,350928980.0,346848980.0,350928980.0,0.063904,NaN,1.960520,NaN,82.185773,218443200.0,-2.436035e+08,53.456667,52.499719
2025-04-30,7090000.0,6.391074e+08,358018980.0,350928980.0,358018980.0,0.060874,NaN,1.699677,NaN,79.572700,379811300.0,-2.432062e+08,53.543333,52.953722
2025-05-01,5260000.0,6.361344e+08,352758980.0,352758980.0,358018980.0,0.034964,NaN,1.574801,NaN,87.473718,281620400.0,-2.435008e+08,53.583333,53.182452
2025-05-02,4990000.0,6.347734e+08,357748980.0,352758980.0,357748980.0,0.031044,NaN,1.629922,NaN,86.818131,269010900.0,-2.400523e+08,53.940000,53.364668


In [5]:
##Dropping column which does not have recent value, which required significant forward prices sereies to calculate
df = df.drop(['DPO_20','ICS_26'],axis=1)

In [6]:
df= df.bfill()

In [7]:
df_ML= df.drop(['close','high','low','volume'],axis=1
               )

In [8]:
df_ML['target'] = (df_ML['open'].shift(-1) > df_ML['open']).astype(int)
#df['target'] = (df['open'].shift(-1) > df['open']).astype(int)


# df_ML.drop('open',axis=1,inplace=True)


In [9]:
df_ML['target'].sum()/df_ML.shape[0]

0.5316070068545317

In [10]:
df_ML.to_clipboard()

In [11]:
def sequential_feature_selection(X: DataFrame, y: Series, var_thresh=0.01, chi2_k='all', lasso_C=0.1):
    # Step 1: Low Variance Filter
    var_selector = VarianceThreshold(threshold=var_thresh)
    X_var = var_selector.fit_transform(X)
    var_cols = X.columns[var_selector.get_support()]
    X = DataFrame(X_var, columns=var_cols)

    # Step 2: Chi-Squared (requires non-negative values)
    X_nonneg = X - X.min()
    chi2_selector = SelectKBest(score_func=chi2, k=chi2_k)
    X_chi2 = chi2_selector.fit_transform(X_nonneg, y)
    chi2_cols = X.columns[chi2_selector.get_support()]
    X = DataFrame(X_chi2, columns=chi2_cols)

    # Step 3: LASSO (Logistic Regression with L1 penalty)
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    lasso = LogisticRegression(penalty='l1', solver='liblinear', C=lasso_C, random_state=42)
    lasso.fit(X_scaled, y)
    lasso_mask = lasso.coef_[0] != 0
    lasso_features = X.columns[lasso_mask]

    return list(lasso_features)

In [12]:
# Seperation of In sample and out of sample train-test data
# 70% of top --> In sample 30%--> out of sample (Test)
n_train = math.floor(0.7*df_ML.shape[0])
df_train =df_ML.iloc[:n_train]
df_test= df_ML.iloc[n_train:]
X_train = df_train.drop('target',axis=1)
X_test = df_test.drop('target',axis=1)
Y_train = df_train['target']
Y_test = df_test['target']

In [13]:
# we will do feature selection as well as traning on the basis of X trainig data set
FS = sequential_feature_selection(X_train,Y_train)
print(FS)

['ABER_ATR_5_15', 'AMATe_LR_8_21_2', 'AMATe_SR_8_21_2', 'OBV', 'OBV_min_2', 'AOBV_LR_2', 'APO_12_26', 'AROOND_14', 'BOP', 'AR_26', 'CDL_DOJI_10_0.1', 'CDL_INSIDE', 'CHOP_14_1_100', 'CMF_20', 'BULLP_13', 'INC_1', 'KVOs_34_55_13', 'MASSI_9_25', 'PSARr_0.02_0.2', 'PSL_12', 'PVOh_12_26_9', 'PVOs_12_26_9', 'PVR', 'PVT', 'SQZ_ON', 'SQZ_OFF', 'SQZPRO_ON_WIDE', 'SQZPRO_ON_NORMAL', 'SQZPRO_OFF', 'STC_10_12_26_0.5', 'SUPERTd_7_3.0', 'THERMOl_20_2_0.5', 'VTXP_14', 'VTXM_14']


In [14]:
### Classification Model 1: Using selected features 
X_train_FS =X_train[FS]
X_test_FS =X_train[FS]

In [16]:
for col in X_train_FS.columns:
    if X_train_FS[col].nunique(dropna=False) == 2:
        X_train_FS.loc[:, col] = X_train_FS[col].astype('category')
        X_test_FS.loc[:, col] = X_test_FS[col].astype('category')  # same column assumed in test


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1838 entries, 2015-01-02 to 2022-03-14
Data columns (total 34 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   ABER_ATR_5_15     1838 non-null   float64 
 1   AMATe_LR_8_21_2   1838 non-null   category
 2   AMATe_SR_8_21_2   1838 non-null   category
 3   OBV               1838 non-null   float64 
 4   OBV_min_2         1838 non-null   float64 
 5   AOBV_LR_2         1838 non-null   category
 6   APO_12_26         1838 non-null   float64 
 7   AROOND_14         1838 non-null   float64 
 8   BOP               1838 non-null   float64 
 9   AR_26             1838 non-null   float64 
 10  CDL_DOJI_10_0.1   1838 non-null   category
 11  CDL_INSIDE        1838 non-null   int64   
 12  CHOP_14_1_100     1838 non-null   float64 
 13  CMF_20            1838 non-null   float64 
 14  BULLP_13          1838 non-null   float64 
 15  INC_1             1838 non-null   category
 16  KVOs_3